In [1]:
from transformers import pipeline, AutoModel, AutoTokenizer,BertTokenizer, BertForMaskedLM, AlbertTokenizer
import pandas as pd
import numpy as np
import os, time
import torch

device = torch.device("cuda:0")
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path = "../../hub/models--Jihuai--bert-ancient-chinese/snapshots/1eb4c2023b541119a81c39be9151ca292f836492/")
model = BertForMaskedLM.from_pretrained(pretrained_model_name_or_path = "../../hub/models--Jihuai--bert-ancient-chinese/snapshots/1eb4c2023b541119a81c39be9151ca292f836492/")
unmasker = pipeline("fill-mask", model = model, tokenizer = tokenizer, device=device)

In [2]:
from core_embed import HookManager, utils
hook_manager = HookManager()

df_query = pd.read_excel("query_ancient_chn.xlsx",index_col=0)
df_query.head()

,Query
1.1,[DESC][TERM]關系者，[MASK]也。
1.2,[DESC][TERM]，為[MASK]之關系也。
2.1,[DESC][TERM]，何以言喻？[MASK]也。
2.2,[DESC][TERM]，何以名状？[MASK]也。
3.1,[DESC][TERM]，要義在諸[MASK]。


In [3]:
df_desc = pd.read_excel('../../1.GPT4_DESC/CHN_ancient/chn_ancient_results_final.xlsx')
df_desc.head()

,古代人际关系名称_现汉,对应的现代人际关系,古代人際關係名稱_文言,description,query
0,暧昧对象之间,暧昧对象之间,曖昧者之間,男女曖昧之事，悖於人倫，是以隱於心而不擅發於外。其或發於外者，亦以含蓄之言，傳微妙之情，假落...,曖昧者之間
1,暗恋对象和暗恋者,暗恋对象-暗恋者,暗戀者與被暗戀者,暗戀者，古語或謂單相思。男女之間，礙于父母之命，媒妁之言，婚姻之有定也，於是己固思慕彼而不敢...,暗戀者與被暗戀者之間
2,报案人和受害者,报案人-受害者,報案者與受害者,報案者之于受害者也，或同系一人，或其鄰裡親朋之知情者也。自秦法以降，遞有閭閻連坐之策，凡知情...,報案者與受害者之間
3,报案人和罪犯,报案人-罪犯,報案者與罪犯,報案者與罪犯之間多對立之關系。報案者舉罪犯之罪，冀以律法懲之。然報案者或有因擿伏發隱而罹罪犯...,報案者與罪犯之間
4,文选(诗集)的编者和作者,编辑-作者,文選之編者與作者,文選詩集之編者與作者，或因敬重欣賞之情，或為功利實在之用。其功利實在之用著，如著者為高官、長...,文選之編者與作者之間


In [4]:
df_desc.iloc[[13,85],:]

,古代人际关系名称_现汉,对应的现代人际关系,古代人際關係名稱_文言,description,query
13,泛泛之交之间,认识但不熟的人之间,泛泛之交之間,泛泛之交，乃交情淺薄之普通關係也。於此關係，彼此不通心意，交遊往來，相待以禮而已。泛泛之交，...,泛泛之交之間
85,普通朋友之间,朋友之间,泛泛之交之間,泛泛之友，可為同好，共論琴棋，共試酒茶，然多止於淺交。所謂淺交者，非親非疏，若敬若重，不甘于...,泛泛之交之間


In [4]:
text = df_query.iloc[3, :]["Query"]
for i, row in df_desc.iloc[:, :].iterrows():
    use_word = text.replace("[DESC]",row['description']).\
               replace("[TERM]",row['query'])
    print(use_word)
    break
unmasker(use_word)

男女曖昧之事，悖於人倫，是以隱於心而不擅發於外。其或發於外者，亦以含蓄之言，傳微妙之情，假落紅秋葉之詩，托香囊繡帕之物，暗致其情，不令外人見也，多是之類也。曖昧者之間，何以名状？[MASK]也。


[{'score': 0.0785604938864708,
  'token': 7403,
  'token_str': '隱',
  'sequence': '男 女 曖 昧 之 事 ， 悖 於 人 倫 ， 是 以 隱 於 心 而 不 擅 發 於 外 。 其 或 發 於 外 者 ， 亦 以 含 蓄 之 言 ， 傳 微 妙 之 情 ， 假 落 紅 秋 葉 之 詩 ， 托 香 囊 繡 帕 之 物 ， 暗 致 其 情 ， 不 令 外 人 見 也 ， 多 是 之 類 也 。 曖 昧 者 之 間 ， 何 以 名 状 ？ 隱 也 。'},
 {'score': 0.06433389335870743,
  'token': 671,
  'token_str': '一',
  'sequence': '男 女 曖 昧 之 事 ， 悖 於 人 倫 ， 是 以 隱 於 心 而 不 擅 發 於 外 。 其 或 發 於 外 者 ， 亦 以 含 蓄 之 言 ， 傳 微 妙 之 情 ， 假 落 紅 秋 葉 之 詩 ， 托 香 囊 繡 帕 之 物 ， 暗 致 其 情 ， 不 令 外 人 見 也 ， 多 是 之 類 也 。 曖 昧 者 之 間 ， 何 以 名 状 ？ 一 也 。'},
 {'score': 0.040295228362083435,
  'token': 2658,
  'token_str': '情',
  'sequence': '男 女 曖 昧 之 事 ， 悖 於 人 倫 ， 是 以 隱 於 心 而 不 擅 發 於 外 。 其 或 發 於 外 者 ， 亦 以 含 蓄 之 言 ， 傳 微 妙 之 情 ， 假 落 紅 秋 葉 之 詩 ， 托 香 囊 繡 帕 之 物 ， 暗 致 其 情 ， 不 令 外 人 見 也 ， 多 是 之 類 也 。 曖 昧 者 之 間 ， 何 以 名 状 ？ 情 也 。'},
 {'score': 0.035906851291656494,
  'token': 2544,
  'token_str': '微',
  'sequence': '男 女 曖 昧 之 事 ， 悖 於 人 倫 ， 是 以 隱 於 心 而 不 擅 發 於 外 。 其 或 發 於 外 者 ， 亦 以 含 蓄 之 言 ， 傳 微 妙 之 情 ， 假 落 紅 秋 葉 之 詩 ，

In [5]:
for query_num, query_row in df_query.iloc[0:2, :].iterrows(): 
    arr_tit = ["CLS", "MASK", "關","系"]
    
    util = utils(arr_tit, unmasker.tokenizer.tokenize)
    text = query_row['Query']
    for v in [1,2]: # no additional requirement, just for create new files
        # output layer
        hook_manager.add_hook(unmasker.model.bert.encoder, util.getHook("embedding_output", v, reset = True))
        # 12 encoder layer
        for i in range(12):
            hook_manager.add_hook(unmasker.model.bert.encoder.layer[i], 
                                  util.getHook("encoder_layer%s" % i, v, reset = True))
        hook_manager.remove_hooks()

    for i, row in df_desc.iloc[:, :].iterrows():
        use_word = text.replace("[DESC]",row['description']).\
                   replace("[TERM]",row['query'])
        print(use_word)

        hook_manager.add_hook(unmasker.model.bert.encoder, util.getHook("embedding_output", 
                              row["query"], use_word))
        # 12 encoder layer
        for i in range(12):
            hook_manager.add_hook(unmasker.model.bert.encoder.layer[i], util.getHook("encoder_layer%s" % i, 
                                                                                     row["query"], use_word))
        res = unmasker(
            use_word
        )
        hook_manager.remove_hooks()
    time.sleep(5)
    os.rename("bert_embedding_data", "bac/q{}_bert_embedding_data".format(query_num))

男女曖昧之事，悖於人倫，是以隱於心而不擅發於外。其或發於外者，亦以含蓄之言，傳微妙之情，假落紅秋葉之詩，托香囊繡帕之物，暗致其情，不令外人見也，多是之類也。曖昧者之間關系者，[MASK]也。
暗戀者，古語或謂單相思。男女之間，礙于父母之命，媒妁之言，婚姻之有定也，於是己固思慕彼而不敢以私意白，彼未知而己切思之，求之不得，其情之至者，所謂“寤寐思服，輾轉反側”是也。其恪于禮者，常以春蠶絲盡、桃花依舊之喻，暗寄其心而不外傳，亦或以暗中護助，不令其有艱困愁苦，而其稍逾制者，則或致佩飾之物也。古之暗戀者，多止乎此，至若季世綱常已墮，男女忽於大防，暗戀轉直，單思為雙，則奔者難禁。暗戀者與被暗戀者之間關系者，[MASK]也。
報案者之于受害者也，或同系一人，或其鄰裡親朋之知情者也。自秦法以降，遞有閭閻連坐之策，凡知情不舉者，按律當坐，王安石所目保甲法，即是也。然世有升降，國有治亂，保甲連坐之策，可行乎令行禁止之時，而文恬武嬉之季，多具文視之耳。且古時刑偵之術寡而限期破案之律在，案犯易脫幹係而有司則損利祿，是以報案者亦須證據確鑿，無可指摘，庶免乎官府之糾擾而人財俱困也。又自董子以來，列朝以儒治國，親親相隱之教，風乎百世，是以近親之間，若非大義滅親之舉，概以不報官府為長，而以鄉約族規處置也。報案者與受害者之間關系者，[MASK]也。
報案者與罪犯之間多對立之關系。報案者舉罪犯之罪，冀以律法懲之。然報案者或有因擿伏發隱而罹罪犯相脅相報之禍者也。又以保甲連坐之製與大義滅親之理，報案者與罪犯之間亦或為親戚鄰裏，此又非必是兩立之勢也。報案者與罪犯之間關系者，[MASK]也。
文選詩集之編者與作者，或因敬重欣賞之情，或為功利實在之用。其功利實在之用著，如著者為高官、長輩、恩師者也，甚者編者或側己文於他人之文，以彰己名也。裁約編選之時，編者常善著者之才華功業。編者亦以其選裁左右讀者之理解。文選詩集之編者與作者，實為古時文人之文化品味、審美觀念、權力關係之鑒鏡也。文選之編者與作者之間關系者，[MASK]也。
表兄弟姐妹者，視同近親，關系親近，家族之情亦深也。彼於童稚之時，可共遊學、相長互助，共解所遇之危難。禮，表兄弟姐妹須循一定之禮節，長愛少，少敬長。及漸長，循授受不親之訓，以其為異姓，可通婚姻故也。表兄弟姐妹之間關系者，[MASK]也。
庖人之與食客者，為服務被服務之關系也。庖人主烹飪，以甘美

C:\Users\DELL\anaconda3\envs\nn\lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


情敵者，二三人傾慕同一對象，遂相競爭。此關係中，有敵意、嫉妬、競爭至於殺心之複雜心緒，然亦或有互尊呼諒者。經史詩賦之中，論及情敵，常有激烈戲謔之矛盾，或高雅而囿於禮法，或直率而流於粗鄙。情敵之間關系者，[MASK]也。
天人之間，乃天地人三才之並立關系，人宜順天地，而與萬物為一。儒道皆謂人當敬天地而順之，於是乎則和也。欲和天人，需通天意，欲通天意，則在貫三才之王者，以儒家正典之論，王者得位聖賢之大號，非權勢之稱也，是以為古代知識階層與君主權力間之製衡也。天人之間關系者，[MASK]也。
人神之間以信仰敬畏為維繫，其方式不離祭祀、祈禱以通達於神靈也，其指歸在乎求得神靈庇佑，解其大惑，脫其苦厄，堅其心志，導其前路也。今之視古，則謂古之信神，原乎科技落後，而驚懼敬畏于自然未知之力量也。然商周以降，中華之地，人以配天，民為神主之論，敬鬼以遠之教，人能弘道之言，天命從德之誡，經籍之載，三教之說，鹹崇人之踐行而斥虛妄之求神也。然則子雖不言怪力亂神，而人之不齊，一致百慮，可導而不可壅，是以君子明辨天理，不惑神鬼，而不責民間之求神問卜以行善避災也。人神之間關系者，[MASK]也。
泛泛之交，乃交情淺薄之普通關係也。於此關係，彼此不通心意，交遊往來，相待以禮而已。泛泛之交，或可謂之朋友，然為關係最疏者也。泛泛之交之間關系者，[MASK]也。
店主與顧客，為買賣交易之雙方也。店主供貨品服務於顧客，顧客以財貨償之。店主須竭其誠以應顧客之需，顧客亦需尊店主之恭敬服務于我也。所謂“買賣不成仁義在”，古時買賣之道，理不逾此。店主與顧客之間關系者，[MASK]也。
官場之尊卑上下，不可逾距。上之於下，等級之中，所差愈多，威勢愈強。是以官員之間，忠敬為首，然忠敬者，忠於所守也，非忠於上司某甲也。上之於下，亦有訓導保全之義，此即古之尊卑有序、和諧中正之道也。然世閒多難行也。官場之上下級之間關系者，[MASK]也。
買賣之夥伴，所共求止於利而已，緣此須相信相助，共分其利。其間信與平最為重，買賣之夥伴須以誠交易，以平分利。彼此亦須行為相協，言語相洽，以求買賣之順遂。買賣之夥伴之間關系者，[MASK]也。
買賣之競爭對手，以欲求多利，欲天下求購者多求於己而俾天下財富多歸於己，於是乎有對手生焉。然生財亦當有道，故相競爭者雖有相爭之欲而求勝於對方，亦皆須守信持中，不得狡求他人之利。要之，亦有尊重也。買賣之競爭

從事買賣者，賣方或以物售之，或以役供之，買方則以貨貝易之。販賣貿買，需重信諾，禁詐紿。賣方所供務必價合質優，買方所償務必踐以約。買者與賣者之間關系者，[MASK]也。
所謂行人，今之路人也，目有所接而言行不必交，以無深交之所需也。或逢於茶樓酒肆，或見於街巷坊裡，非有際會因緣，相交不過泛泛而已。行人之交多無深交厚誼可溯，然疏而有禮，遠而存敬。行人之間關系者，[MASK]也。
狐朋狗友，品行不檢而相偕作樂取悅心欲者也，所好皆賭博、酗飲、狎妓之陋習。狐朋狗友之交，觥籌交錯、宴酣之樂顯於外，而犬馬聲色、熏心色欲隱其中也。狐朋狗黨之間關系者，[MASK]也。
泛泛之友，可為同好，共論琴棋，共試酒茶，然多止於淺交。所謂淺交者，非親非疏，若敬若重，不甘于醴，亦非純淡如水也，以未深交之故也。泛泛之交之間關系者，[MASK]也。
故舊者，情誼深遠之友也。故舊之友多共曆往事種種，然溫不增華，寒不改葉，能四時而不衰，歷險夷而仍固，相輔相助，情堅若金。故舊之間關系者，[MASK]也。
摯友者，無血緣之親，然勝乎同產，雖貧富貴賤、生死離別不能阻隔之。摯友相交，以拳拳之心同攜手、共患難，實無價之友誼也。摯友之間關系者，[MASK]也。
友誼之至境是謂知己。其情之深厚，遠勝泛泛相交，亦相異於親子牽絆，如奏高山流水于知音也，意動即神會，勝卻言語無數。互為知己者，彼之喜怒哀樂皆可歷歷於己心，至情至性，合若符契，而相敬之以禮；若得一知己，遑論肝膽相照，雖九死其猶未悔。友誼之至境，不過是也。知己之間關系者，[MASK]也。
乞者與行者，多遇於市肆。乞者所欲者惟行者之施，行者之施亦可彰己憐憫之心。然二者相遇，行者多無所施，乞者或無所得，以不必施故也。乞丐與行人之間關系者，[MASK]也。
新故官吏之閒，權力交接之關係也。去者有升、謫、免、致仕之別，而新任者，生疏于此間府衙事務，故舊任有授新任以經驗故事之責。然則離任者或以親近者安排新任左右，雖去任而威信存，且交接之際，多涉賦稅度支之核查、檔案文書之清點，案件審理之清白，田地戶口之數目，是以輕則生隙，重則黨伐。新故吏之間關系者，[MASK]也。
獄卒之于囚犯，有羈押之權，囚犯受制于獄卒，受監守之限制。獄卒待囚犯何如，視囚犯家族之貴賤、案情判罰之輕重，及親友打點之多少而各異也。若利益相合，或義氣相投，獄卒囚犯亦可結友，然為獄卒者疾言厲色、受其暴虐者多為常態。獄

雙胞胎因其母一胎同產之同胞，其情於血親手足之中為最近也。同孕母腹，同生一時，同長一處，其形其態其跡在成人之前尤多為相似，故其未冠笄之時，蓋情最深厚，或施及平生。雙胞胎之間，為[MASK]之關系也。
於政治之中，兩方之會晤多因權與利之爭鬥也。多以其政治與國家所屬不同，冀會晤以求成結盟。兩方須守盟會之理，於守自身之利為上為底，亦宜慮對方之利，以成盟約也。政治談判中對立之雙方之間，為[MASK]之關系也。
堂兄弟姐妹為親戚，然其親不過於同胞兄弟姐妹。於家族之中，堂兄弟姐妹可與嫡子女同長共學，亦同為長者所愛，然於家財分配，若非嫡子女亡故出繼，則不得分其家財，然亦多有倚勢強奪家財之事。論及尊卑，堂兄弟姐妹之長者亦為家族所重。堂兄弟姐妹之間，為[MASK]之關系也。
古者以父系別家族，故同父異母與同母異父之子女之所受對待全然不同。若子女為同父異母，則其於家族之中尊卑貴賤則從母之所處為父之發妻、續弦、妾之等而不同，及其母之存亡與母家之勢而定，然其爲一家之人，則無疑也。若子女為同母異父，則多因父之續娶他婦，其母之前夫子女多為後嫁之家族異姓視之，其被賤待，亦其常也，然亦須視其後父家之心與其母家之勢。同父異母與同母異父之子女之間，為[MASK]之關系也。
繼子女之間，雖為親戚，但彼此犬牙交錯，一言難磬。繼母與繼父之子女之關系亦相去萬裏。古者，父親多主家族，父親續娶，繼母亦為繼子之母也。然若母親改醮，繼子則從母而入繼父家，其家族關系亦全然改換也。若父母於其前夫前妻亡故後另結婚姻，則或因繼父繼母之愛而另生齟齬。家產之分配，若其母為繼母，則繼子女（若其女子未嫁）以律法可得所應之家產。若其父為繼父，則繼子女非繼父及家族所允，以律法不得受家產。繼子女之間，為[MASK]之關系也。
古之為官，同僚之間以官位相近，同朝合堂，共謀於事。然或因其職所守相近，未免於相爭，而生齟齬者亦多有之。同僚之間尊卑之序則定以官位高低、資望長短。同僚之間，為[MASK]之關系也。
庠序之間，同窗之間以求學問知識為共識，相助於學，共進於道。然因修業之優劣，成績之高下，而生齟齬者亦有之。同窗之間，常為對等之交，然亦可守尊卑之禮而以年齒為序，此多為交往之禮便耳。同窗之間，為[MASK]之關系也。
所謂搭訕者，意在親近被搭訕者，以其欲與被搭訕者交好而有所圖也。搭訕者之於被搭訕者，或欲作綈袍之義，或欲結琴瑟之好，或欲為買賣之宜。

遠親者，有血緣之絆，無近鄰之切，往來問候多訴諸婚喪嫁娶節氣，寄乎禮法倫常，以厚親戚之情。然澆薄之世，如涉祖業田產之爭，則雖遠親亦或生齟齬。遠親之間，為[MASK]之關系也。
子婿之於岳家，理應互敬互賴，然岳家垂青與否，多決於子婿之功名、族望。除入贅而改姓易族者，翁婿雖名為父子，實各異家，其情弗如同簷之婆媳遠甚，遑論血緣之親。岳家與子婿之間，為[MASK]之關系也。
同袍之誼，多喻軍旅士伍心心相惜。士卒征戰於沙場，同浴血汗、共赴國難，皆懷身先致死、盡忠報國之念。詩曰：豈曰無衣，與子同袍。是以同袍之親，勝似兄弟。同袍之間，為[MASK]之關系也。
官民之間，上下領屬之關係也。為官之道，必除害興利，善度民力，正風立綱；為民之要，必遵紀守法，立身行道。官民相處亦如父子，為人父則慈，為人子則孝，為人上則明，為人下則聖，父慈子孝，上明下聖，治之紀也。官民之間，為[MASK]之關系也。
結黨而敵對者，始於小人之同利，及其見利則爭先，或利盡而交疏，乃反相賊害。結黨者之利異，多為政見、觀念相抵，至成水火之勢。為求己利，雖其親戚兄弟不保也。敵對黨派之間，為[MASK]之關系也。
同盟即志同道合者。君子與君子以同道為朋，共守道義，行忠信，惜名節。是故同盟者得以守望相助，進退如一，莫逆於心。同盟黨派之間，為[MASK]之關系也。
保人者，受託于雙方顧主，牽繫買賣關係，以局外之身處理其間事務者也。顧主雙方多聘保人以求交易之公正，保人亦從中牟利。保人之制，早至魏晉隋唐已有所載，古商貿制度之完善據此可窺一二。保人與顧主之間，為[MASK]之關系也。
所謂忠臣，效忠于社稷者也，以鞠躬盡瘁、死而後已為己任，多為世之表率，歷代稱頌不絕。所謂逆賊，離君叛國者也，陷社稷於累卵之危、倒懸之急，世人為之不齒。忠臣與逆賊之間，為[MASK]之關系也。
主客之交，以互敬互重為本。為人主者，待人接物需完備周致，以求賓至如歸；為賓客者，亦需禮尚往來，以示謝忱。古之人倫禮制，亦見於主客相交之道也。主人與賓客之間，為[MASK]之關系也。
主奴者，主人為主而女婢為從也。主人有令，奴必應之，不可不從，以其必終於主人故也。奴之衣食起居，身家性命，全仰主意。主人和善體恤，則奴亦可豐足；主慳吝刻薄，則奴亦常縈困頓。然奴之私忠於其主也，不必係於主之慈暴，以世論其忠若犬故也。主奴之間，為[MASK]之關系也。
乳母之與嬰孩之家長，僱

In [6]:
for query_num, query_row in df_query.iloc[2:, :].iterrows(): 
    arr_tit = ["CLS", "MASK"]
    
    util = utils(arr_tit, unmasker.tokenizer.tokenize)
    text = query_row['Query']
    for v in [1,2]: # no additional requirement, just for create new files
        # output layer
        hook_manager.add_hook(unmasker.model.bert.encoder, util.getHook("embedding_output", v, reset = True))
        # 12 encoder layer
        for i in range(12):
            hook_manager.add_hook(unmasker.model.bert.encoder.layer[i], 
                                  util.getHook("encoder_layer%s" % i, v, reset = True))
        hook_manager.remove_hooks()

    for i, row in df_desc.iloc[:, :].iterrows():
        use_word = text.replace("[DESC]",row['description']).\
                   replace("[TERM]",row['query'])
        print(use_word)

        hook_manager.add_hook(unmasker.model.bert.encoder, util.getHook("embedding_output", 
                              row["query"], use_word))
        # 12 encoder layer
        for i in range(12):
            hook_manager.add_hook(unmasker.model.bert.encoder.layer[i], util.getHook("encoder_layer%s" % i, 
                                                                                     row["query"], use_word))
        res = unmasker(
            use_word
        )
        hook_manager.remove_hooks()
    time.sleep(5)
    os.rename("bert_embedding_data", "bac/q{}_bert_embedding_data".format(query_num))

男女曖昧之事，悖於人倫，是以隱於心而不擅發於外。其或發於外者，亦以含蓄之言，傳微妙之情，假落紅秋葉之詩，托香囊繡帕之物，暗致其情，不令外人見也，多是之類也。曖昧者之間，何以言喻？[MASK]也。
暗戀者，古語或謂單相思。男女之間，礙于父母之命，媒妁之言，婚姻之有定也，於是己固思慕彼而不敢以私意白，彼未知而己切思之，求之不得，其情之至者，所謂“寤寐思服，輾轉反側”是也。其恪于禮者，常以春蠶絲盡、桃花依舊之喻，暗寄其心而不外傳，亦或以暗中護助，不令其有艱困愁苦，而其稍逾制者，則或致佩飾之物也。古之暗戀者，多止乎此，至若季世綱常已墮，男女忽於大防，暗戀轉直，單思為雙，則奔者難禁。暗戀者與被暗戀者之間，何以言喻？[MASK]也。
報案者之于受害者也，或同系一人，或其鄰裡親朋之知情者也。自秦法以降，遞有閭閻連坐之策，凡知情不舉者，按律當坐，王安石所目保甲法，即是也。然世有升降，國有治亂，保甲連坐之策，可行乎令行禁止之時，而文恬武嬉之季，多具文視之耳。且古時刑偵之術寡而限期破案之律在，案犯易脫幹係而有司則損利祿，是以報案者亦須證據確鑿，無可指摘，庶免乎官府之糾擾而人財俱困也。又自董子以來，列朝以儒治國，親親相隱之教，風乎百世，是以近親之間，若非大義滅親之舉，概以不報官府為長，而以鄉約族規處置也。報案者與受害者之間，何以言喻？[MASK]也。
報案者與罪犯之間多對立之關系。報案者舉罪犯之罪，冀以律法懲之。然報案者或有因擿伏發隱而罹罪犯相脅相報之禍者也。又以保甲連坐之製與大義滅親之理，報案者與罪犯之間亦或為親戚鄰裏，此又非必是兩立之勢也。報案者與罪犯之間，何以言喻？[MASK]也。
文選詩集之編者與作者，或因敬重欣賞之情，或為功利實在之用。其功利實在之用著，如著者為高官、長輩、恩師者也，甚者編者或側己文於他人之文，以彰己名也。裁約編選之時，編者常善著者之才華功業。編者亦以其選裁左右讀者之理解。文選詩集之編者與作者，實為古時文人之文化品味、審美觀念、權力關係之鑒鏡也。文選之編者與作者之間，何以言喻？[MASK]也。
表兄弟姐妹者，視同近親，關系親近，家族之情亦深也。彼於童稚之時，可共遊學、相長互助，共解所遇之危難。禮，表兄弟姐妹須循一定之禮節，長愛少，少敬長。及漸長，循授受不親之訓，以其為異姓，可通婚姻故也。表兄弟姐妹之間，何以言喻？[MASK]也。
庖人之與食客者，為服務被服務之關

戲子，上臺演出者也；看官，觀賞表演者也。戲子以其表演技巧取悅看客，博看官捧場，而看官之回應，或於大快人心之處喝采，或於角色含冤負屈之處泣下。高超之技藝固得掌聲口譽，反之則致倒彩譏嘲。因之戲子須重看官之反應，然戲子卑微低賤，看官之於戲子，決於看官自身之態度與修養，借其賤戲子，以之為下等，亦不違世俗之德。戲子與看官之間，何以言喻？[MASK]也。
音樂曲藝人，以賞心悅耳之音求生計者也；聽眾，品賞其音樂曲藝者也。音樂曲藝人以其曲藝之才悅聽眾之耳，聽眾則據藝人之水準報以讚譽財物也。要之，藝人須以聽眾之反應為重，然曲藝人恆処賤籍，故與聽眾或不對等也。音樂曲藝人與聽眾之間，何以言喻？[MASK]也。
祖孫之間尊卑長幼宜井然有序。祖父母位高於父母，是以孫子女之於祖父母，尤須以禮敬重孝順之，以終其天年，是亦儒家倫理之不可或缺者也。祖孫之間，何以言喻？[MASK]也。
大夫，醫者之敬稱也。古之良大夫也，以扁鵲華佗之資，行救死扶傷之舉。病者若知曉大夫之醫術兼信之，則病者得瘳而醫者受譽，然則病者之與醫者，亦隨醫者口碑之優劣、人品之高下，與夫病者病情之變化、性格之差異而有別。大夫與病者之間，何以言喻？[MASK]也。
姨父姨母與侄子侄女之間存親戚之情也。姨父母以其長者慈顧幼小，幼者敬其長者，此為華夏家庭親情倫理之恆常。姨父姨母與侄子侄女之間，何以言喻？[MASK]也。
行俠仗義者，救受助者於水火災厄之中。仗其勇力，憑其志氣，赴人之困頓，軌俠之道義。至於受助者，則當銜環結草以報其恩也，是俠氣之所傳，而仁義有所弘也，夫緩人之所急，脫人之所危，曷可少也。行俠仗義者與受助者之間，何以言喻？[MASK]也。
妓女者，誤落風塵，陷於倡家，有姿色者賓客如市而衰老者落拓餘生。嫖客以錢財之豐厚，走馬章台，縱其情欲也。然二者之間，或漸生情愫，以成好合者，往往有之，然沉箱剃髮之悲，蓋尤其多也。狎妓違君子德風之誨，悖於人倫常理，為世之所誅責。然歷代官家律令以存秦樓楚館為慣例，《管子》發其端也。嫖客與妓女之間，何以言喻？[MASK]也。
掌櫃者，受店主所托，掌日常經營之事宜，凡貨物之進出、價格之定立、賬目之記錄諸類者皆須過問。掌櫃當不負店主所托之責，店主則當行察看之勸，以驗其實績。掌櫃與店主之間，何以言喻？[MASK]也。
夥計與掌櫃之間，上下級也。夥計，掌櫃之下屬也，掌日常營業之細事，凡接待賓客、售賣商品、搬運貨

施恩者與受恩者為恩義與感恩之關係。施恩者之助受恩者也，或以善行，或以援手，受恩者遂感念其德，雖滴水之恩，木瓜之贈，亦矢報以湧泉瓊瑤也。古以'知恩圖報'為德之大者也。施恩者與受恩者之間，何以名状？[MASK]也。
青梅竹馬，即為男女自幼同遊嬉戲，心生情愫者也。其情純真深沉，或終為伉儷，或為知己。古時文人墨客之述此情者，至純至美，感人心脾。青梅竹馬之間，何以名状？[MASK]也。
情敵者，二三人傾慕同一對象，遂相競爭。此關係中，有敵意、嫉妬、競爭至於殺心之複雜心緒，然亦或有互尊呼諒者。經史詩賦之中，論及情敵，常有激烈戲謔之矛盾，或高雅而囿於禮法，或直率而流於粗鄙。情敵之間，何以名状？[MASK]也。
天人之間，乃天地人三才之並立關系，人宜順天地，而與萬物為一。儒道皆謂人當敬天地而順之，於是乎則和也。欲和天人，需通天意，欲通天意，則在貫三才之王者，以儒家正典之論，王者得位聖賢之大號，非權勢之稱也，是以為古代知識階層與君主權力間之製衡也。天人之間，何以名状？[MASK]也。
人神之間以信仰敬畏為維繫，其方式不離祭祀、祈禱以通達於神靈也，其指歸在乎求得神靈庇佑，解其大惑，脫其苦厄，堅其心志，導其前路也。今之視古，則謂古之信神，原乎科技落後，而驚懼敬畏于自然未知之力量也。然商周以降，中華之地，人以配天，民為神主之論，敬鬼以遠之教，人能弘道之言，天命從德之誡，經籍之載，三教之說，鹹崇人之踐行而斥虛妄之求神也。然則子雖不言怪力亂神，而人之不齊，一致百慮，可導而不可壅，是以君子明辨天理，不惑神鬼，而不責民間之求神問卜以行善避災也。人神之間，何以名状？[MASK]也。
泛泛之交，乃交情淺薄之普通關係也。於此關係，彼此不通心意，交遊往來，相待以禮而已。泛泛之交，或可謂之朋友，然為關係最疏者也。泛泛之交之間，何以名状？[MASK]也。
店主與顧客，為買賣交易之雙方也。店主供貨品服務於顧客，顧客以財貨償之。店主須竭其誠以應顧客之需，顧客亦需尊店主之恭敬服務于我也。所謂“買賣不成仁義在”，古時買賣之道，理不逾此。店主與顧客之間，何以名状？[MASK]也。
官場之尊卑上下，不可逾距。上之於下，等級之中，所差愈多，威勢愈強。是以官員之間，忠敬為首，然忠敬者，忠於所守也，非忠於上司某甲也。上之於下，亦有訓導保全之義，此即古之尊卑有序、和諧中正之道也。然世閒多難行也。官場之上下級之間，何以名状？[MA

家長敬重信任其子女之師也，是以教育之任托之。師長傳道受業解惑，以誨其子弟。總言之，師長位高權重，掌教學之法諸多事宜，故家長須聽而從之。彼此或以嚴父之姿共擔子弟教育之責。師長與家長之間，何以名状？[MASK]也。
同鄉之間往來密切，彼此相助，共解困厄。同鄉之間，同飲一鄉之水，同言一方之音，風土人情共相習知，故他鄉之逢，倍感親切。同鄉之間，何以名状？[MASK]也。
鄰里相處之道，可奠鄉里秩序之基。鄰里者，共助、共用、共慶，其要在乎協同互敬，講信修睦，可築鄉里之大同。結姻親、共耕織、連什伍，皆可固鄰里。鄰裡之間，何以名状？[MASK]也。
相與出行者，同途之際，經歷相同，利益相干而目的相通也。故可相持互助，以應行途之變。同遊關係之要，在諸同心同德，共克困艱。相與出行者之間，何以名状？[MASK]也。
從事買賣者，賣方或以物售之，或以役供之，買方則以貨貝易之。販賣貿買，需重信諾，禁詐紿。賣方所供務必價合質優，買方所償務必踐以約。買者與賣者之間，何以名状？[MASK]也。
所謂行人，今之路人也，目有所接而言行不必交，以無深交之所需也。或逢於茶樓酒肆，或見於街巷坊裡，非有際會因緣，相交不過泛泛而已。行人之交多無深交厚誼可溯，然疏而有禮，遠而存敬。行人之間，何以名状？[MASK]也。
狐朋狗友，品行不檢而相偕作樂取悅心欲者也，所好皆賭博、酗飲、狎妓之陋習。狐朋狗友之交，觥籌交錯、宴酣之樂顯於外，而犬馬聲色、熏心色欲隱其中也。狐朋狗黨之間，何以名状？[MASK]也。
泛泛之友，可為同好，共論琴棋，共試酒茶，然多止於淺交。所謂淺交者，非親非疏，若敬若重，不甘于醴，亦非純淡如水也，以未深交之故也。泛泛之交之間，何以名状？[MASK]也。
故舊者，情誼深遠之友也。故舊之友多共曆往事種種，然溫不增華，寒不改葉，能四時而不衰，歷險夷而仍固，相輔相助，情堅若金。故舊之間，何以名状？[MASK]也。
摯友者，無血緣之親，然勝乎同產，雖貧富貴賤、生死離別不能阻隔之。摯友相交，以拳拳之心同攜手、共患難，實無價之友誼也。摯友之間，何以名状？[MASK]也。
友誼之至境是謂知己。其情之深厚，遠勝泛泛相交，亦相異於親子牽絆，如奏高山流水于知音也，意動即神會，勝卻言語無數。互為知己者，彼之喜怒哀樂皆可歷歷於己心，至情至性，合若符契，而相敬之以禮；若得一知己，遑論肝膽相照，雖九死其猶未悔。友誼之至境，不過是

百工之師父與學徒，在於技藝之傳承。傳技之時，師者須教導訓誡其徒，徒者須尊敬愛重其師。其間尊與忠為最重，徒須尊師忠命，師須護持教徒。師傅與學徒之間，要義在諸[MASK]。
古之掌櫃與客人，因買賣交易為緣。掌櫃者經營其店。客人者消費之人也，求店主之所供。掌櫃須供客人所需，以求信與利。客人亦須敬重掌櫃，不得違其買賣之道。掌櫃與客人之間，要義在諸[MASK]。
古之首領與反賊，以勢不兩雄為常。首領者，為國之首領，反賊者，為國之叛逆。首領須守其威勢與現有之秩序，反賊則欲反叛其威勢與現有之秩序。首領與反賊之間，要義在諸[MASK]。
叔侄為血親。叔者為父之弟，侄者為兄之子。叔有教育侄之責，侄須愛敬叔，然亦須視其父母之存亡與學識尊卑論叔撫育之任也。叔侄關係於家族之中為極重要之關係。叔侄之間，要義在諸[MASK]。
雙胞胎因其母一胎同產之同胞，其情於血親手足之中為最近也。同孕母腹，同生一時，同長一處，其形其態其跡在成人之前尤多為相似，故其未冠笄之時，蓋情最深厚，或施及平生。雙胞胎之間，要義在諸[MASK]。
於政治之中，兩方之會晤多因權與利之爭鬥也。多以其政治與國家所屬不同，冀會晤以求成結盟。兩方須守盟會之理，於守自身之利為上為底，亦宜慮對方之利，以成盟約也。政治談判中對立之雙方之間，要義在諸[MASK]。
堂兄弟姐妹為親戚，然其親不過於同胞兄弟姐妹。於家族之中，堂兄弟姐妹可與嫡子女同長共學，亦同為長者所愛，然於家財分配，若非嫡子女亡故出繼，則不得分其家財，然亦多有倚勢強奪家財之事。論及尊卑，堂兄弟姐妹之長者亦為家族所重。堂兄弟姐妹之間，要義在諸[MASK]。
古者以父系別家族，故同父異母與同母異父之子女之所受對待全然不同。若子女為同父異母，則其於家族之中尊卑貴賤則從母之所處為父之發妻、續弦、妾之等而不同，及其母之存亡與母家之勢而定，然其爲一家之人，則無疑也。若子女為同母異父，則多因父之續娶他婦，其母之前夫子女多為後嫁之家族異姓視之，其被賤待，亦其常也，然亦須視其後父家之心與其母家之勢。同父異母與同母異父之子女之間，要義在諸[MASK]。
繼子女之間，雖為親戚，但彼此犬牙交錯，一言難磬。繼母與繼父之子女之關系亦相去萬裏。古者，父親多主家族，父親續娶，繼母亦為繼子之母也。然若母親改醮，繼子則從母而入繼父家，其家族關系亦全然改換也。若父母於其前夫前妻亡故後另結婚姻，則或因繼父繼母之愛而

...

山長者，書院衆師之長，亦書院之主管也；講席，書院所延聘之教師也。書院之治，向以寬而有別於官學也。官學之治，以屬官府而有等級高下之判。書院則否，以山長之道德文章為維繫也，以地方劃撥之田地房舍為生養也，是以山長之與講席，一講席之首座耳，志同則合，志異則散，山長之管書院者，攝也，固當共尊書院之規，不以勢相凌，是以寬。然有清之後，書院亦官學化也，非概得以寬一之。學府書院之山長與講席之間，多[MASK]。
同父同母之子女，為同胞之兄弟姊妹也，此乃血緣至親。同胞兄弟姊妹間多親呢，常勝於非同胞者。同父同母之子女之間，多[MASK]。
戲班與劇作者之間多合作也。劇作者撰作戲劇劇本，而戲班搬演劇作，以供觀眾欣賞。戲班之人身卑籍賤，然劇作者或為讀書人，位四民之首，甚者有功名焉，以差距懸殊之故，所謂合作者，亦鬆散之形式也。劇作者未必依演出之狀況而作成案頭之曲，戲班之於作品之呈現，亦未必符契原文，其違作者原意而有自行發揮之處者，蓋不勝舉也。戲班與劇作者之間，多[MASK]。
戲子，上臺演出者也；看官，觀賞表演者也。戲子以其表演技巧取悅看客，博看官捧場，而看官之回應，或於大快人心之處喝采，或於角色含冤負屈之處泣下。高超之技藝固得掌聲口譽，反之則致倒彩譏嘲。因之戲子須重看官之反應，然戲子卑微低賤，看官之於戲子，決於看官自身之態度與修養，借其賤戲子，以之為下等，亦不違世俗之德。戲子與看官之間，多[MASK]。
音樂曲藝人，以賞心悅耳之音求生計者也；聽眾，品賞其音樂曲藝者也。音樂曲藝人以其曲藝之才悅聽眾之耳，聽眾則據藝人之水準報以讚譽財物也。要之，藝人須以聽眾之反應為重，然曲藝人恆処賤籍，故與聽眾或不對等也。音樂曲藝人與聽眾之間，多[MASK]。
祖孫之間尊卑長幼宜井然有序。祖父母位高於父母，是以孫子女之於祖父母，尤須以禮敬重孝順之，以終其天年，是亦儒家倫理之不可或缺者也。祖孫之間，多[MASK]。
大夫，醫者之敬稱也。古之良大夫也，以扁鵲華佗之資，行救死扶傷之舉。病者若知曉大夫之醫術兼信之，則病者得瘳而醫者受譽，然則病者之與醫者，亦隨醫者口碑之優劣、人品之高下，與夫病者病情之變化、性格之差異而有別。大夫與病者之間，多[MASK]。
姨父姨母與侄子侄女之間存親戚之情也。姨父母以其長者慈顧幼小，幼者敬其長者，此為華夏家庭親情倫理之恆常。姨父姨母與侄子侄女之間，多[MASK]。
行俠仗義者，救受助者於

仇敵之間乃對立衝突之關係也。敵者，匹敵也，論其才智，則相儔類，而或以利益之不均、或以地位之不匹、或以權力之不及、或以忠誠之不貞，諸此之屬，心生怨望而欲戕害以快其心者，仇敵之事也。然則，仇敵既以相噬為常，亦以其相匹，而或生相惜之感，且古時人分等級，同匹儔之人，其未為仇敵之初，或有通家之好，秦晉之交而同窗之誼也，是以古之仇敵，視今日為複雜也。故經史詩賦之中，感概仇敵之才智氣節者，亦不知凡幾也。仇敵之間，視若[MASK]。
施恩者與受恩者為恩義與感恩之關係。施恩者之助受恩者也，或以善行，或以援手，受恩者遂感念其德，雖滴水之恩，木瓜之贈，亦矢報以湧泉瓊瑤也。古以'知恩圖報'為德之大者也。施恩者與受恩者之間，視若[MASK]。
青梅竹馬，即為男女自幼同遊嬉戲，心生情愫者也。其情純真深沉，或終為伉儷，或為知己。古時文人墨客之述此情者，至純至美，感人心脾。青梅竹馬之間，視若[MASK]。
情敵者，二三人傾慕同一對象，遂相競爭。此關係中，有敵意、嫉妬、競爭至於殺心之複雜心緒，然亦或有互尊呼諒者。經史詩賦之中，論及情敵，常有激烈戲謔之矛盾，或高雅而囿於禮法，或直率而流於粗鄙。情敵之間，視若[MASK]。
天人之間，乃天地人三才之並立關系，人宜順天地，而與萬物為一。儒道皆謂人當敬天地而順之，於是乎則和也。欲和天人，需通天意，欲通天意，則在貫三才之王者，以儒家正典之論，王者得位聖賢之大號，非權勢之稱也，是以為古代知識階層與君主權力間之製衡也。天人之間，視若[MASK]。
人神之間以信仰敬畏為維繫，其方式不離祭祀、祈禱以通達於神靈也，其指歸在乎求得神靈庇佑，解其大惑，脫其苦厄，堅其心志，導其前路也。今之視古，則謂古之信神，原乎科技落後，而驚懼敬畏于自然未知之力量也。然商周以降，中華之地，人以配天，民為神主之論，敬鬼以遠之教，人能弘道之言，天命從德之誡，經籍之載，三教之說，鹹崇人之踐行而斥虛妄之求神也。然則子雖不言怪力亂神，而人之不齊，一致百慮，可導而不可壅，是以君子明辨天理，不惑神鬼，而不責民間之求神問卜以行善避災也。人神之間，視若[MASK]。
泛泛之交，乃交情淺薄之普通關係也。於此關係，彼此不通心意，交遊往來，相待以禮而已。泛泛之交，或可謂之朋友，然為關係最疏者也。泛泛之交之間，視若[MASK]。
店主與顧客，為買賣交易之雙方也。店主供貨品服務於顧客，顧客以財貨償之。店主須竭其誠以應顧

鄰里相處之道，可奠鄉里秩序之基。鄰里者，共助、共用、共慶，其要在乎協同互敬，講信修睦，可築鄉里之大同。結姻親、共耕織、連什伍，皆可固鄰里。鄰裡之間，視若[MASK]。
相與出行者，同途之際，經歷相同，利益相干而目的相通也。故可相持互助，以應行途之變。同遊關係之要，在諸同心同德，共克困艱。相與出行者之間，視若[MASK]。
從事買賣者，賣方或以物售之，或以役供之，買方則以貨貝易之。販賣貿買，需重信諾，禁詐紿。賣方所供務必價合質優，買方所償務必踐以約。買者與賣者之間，視若[MASK]。
所謂行人，今之路人也，目有所接而言行不必交，以無深交之所需也。或逢於茶樓酒肆，或見於街巷坊裡，非有際會因緣，相交不過泛泛而已。行人之交多無深交厚誼可溯，然疏而有禮，遠而存敬。行人之間，視若[MASK]。
狐朋狗友，品行不檢而相偕作樂取悅心欲者也，所好皆賭博、酗飲、狎妓之陋習。狐朋狗友之交，觥籌交錯、宴酣之樂顯於外，而犬馬聲色、熏心色欲隱其中也。狐朋狗黨之間，視若[MASK]。
泛泛之友，可為同好，共論琴棋，共試酒茶，然多止於淺交。所謂淺交者，非親非疏，若敬若重，不甘于醴，亦非純淡如水也，以未深交之故也。泛泛之交之間，視若[MASK]。
故舊者，情誼深遠之友也。故舊之友多共曆往事種種，然溫不增華，寒不改葉，能四時而不衰，歷險夷而仍固，相輔相助，情堅若金。故舊之間，視若[MASK]。
摯友者，無血緣之親，然勝乎同產，雖貧富貴賤、生死離別不能阻隔之。摯友相交，以拳拳之心同攜手、共患難，實無價之友誼也。摯友之間，視若[MASK]。
友誼之至境是謂知己。其情之深厚，遠勝泛泛相交，亦相異於親子牽絆，如奏高山流水于知音也，意動即神會，勝卻言語無數。互為知己者，彼之喜怒哀樂皆可歷歷於己心，至情至性，合若符契，而相敬之以禮；若得一知己，遑論肝膽相照，雖九死其猶未悔。友誼之至境，不過是也。知己之間，視若[MASK]。
乞者與行者，多遇於市肆。乞者所欲者惟行者之施，行者之施亦可彰己憐憫之心。然二者相遇，行者多無所施，乞者或無所得，以不必施故也。乞丐與行人之間，視若[MASK]。
新故官吏之閒，權力交接之關係也。去者有升、謫、免、致仕之別，而新任者，生疏于此間府衙事務，故舊任有授新任以經驗故事之責。然則離任者或以親近者安排新任左右，雖去任而威信存，且交接之際，多涉賦稅度支之核查、檔案文書之清點，案件審理之清白，

雙胞胎因其母一胎同產之同胞，其情於血親手足之中為最近也。同孕母腹，同生一時，同長一處，其形其態其跡在成人之前尤多為相似，故其未冠笄之時，蓋情最深厚，或施及平生。雙胞胎之間，彼此[MASK]。
於政治之中，兩方之會晤多因權與利之爭鬥也。多以其政治與國家所屬不同，冀會晤以求成結盟。兩方須守盟會之理，於守自身之利為上為底，亦宜慮對方之利，以成盟約也。政治談判中對立之雙方之間，彼此[MASK]。
堂兄弟姐妹為親戚，然其親不過於同胞兄弟姐妹。於家族之中，堂兄弟姐妹可與嫡子女同長共學，亦同為長者所愛，然於家財分配，若非嫡子女亡故出繼，則不得分其家財，然亦多有倚勢強奪家財之事。論及尊卑，堂兄弟姐妹之長者亦為家族所重。堂兄弟姐妹之間，彼此[MASK]。
古者以父系別家族，故同父異母與同母異父之子女之所受對待全然不同。若子女為同父異母，則其於家族之中尊卑貴賤則從母之所處為父之發妻、續弦、妾之等而不同，及其母之存亡與母家之勢而定，然其爲一家之人，則無疑也。若子女為同母異父，則多因父之續娶他婦，其母之前夫子女多為後嫁之家族異姓視之，其被賤待，亦其常也，然亦須視其後父家之心與其母家之勢。同父異母與同母異父之子女之間，彼此[MASK]。
繼子女之間，雖為親戚，但彼此犬牙交錯，一言難磬。繼母與繼父之子女之關系亦相去萬裏。古者，父親多主家族，父親續娶，繼母亦為繼子之母也。然若母親改醮，繼子則從母而入繼父家，其家族關系亦全然改換也。若父母於其前夫前妻亡故後另結婚姻，則或因繼父繼母之愛而另生齟齬。家產之分配，若其母為繼母，則繼子女（若其女子未嫁）以律法可得所應之家產。若其父為繼父，則繼子女非繼父及家族所允，以律法不得受家產。繼子女之間，彼此[MASK]。
古之為官，同僚之間以官位相近，同朝合堂，共謀於事。然或因其職所守相近，未免於相爭，而生齟齬者亦多有之。同僚之間尊卑之序則定以官位高低、資望長短。同僚之間，彼此[MASK]。
庠序之間，同窗之間以求學問知識為共識，相助於學，共進於道。然因修業之優劣，成績之高下，而生齟齬者亦有之。同窗之間，常為對等之交，然亦可守尊卑之禮而以年齒為序，此多為交往之禮便耳。同窗之間，彼此[MASK]。
所謂搭訕者，意在親近被搭訕者，以其欲與被搭訕者交好而有所圖也。搭訕者之於被搭訕者，或欲作綈袍之義，或欲結琴瑟之好，或欲為買賣之宜。二者之關系，主動與被動並存，不確與不穩共生

官民之間，上下領屬之關係也。為官之道，必除害興利，善度民力，正風立綱；為民之要，必遵紀守法，立身行道。官民相處亦如父子，為人父則慈，為人子則孝，為人上則明，為人下則聖，父慈子孝，上明下聖，治之紀也。官民之間，彼此[MASK]。
結黨而敵對者，始於小人之同利，及其見利則爭先，或利盡而交疏，乃反相賊害。結黨者之利異，多為政見、觀念相抵，至成水火之勢。為求己利，雖其親戚兄弟不保也。敵對黨派之間，彼此[MASK]。
同盟即志同道合者。君子與君子以同道為朋，共守道義，行忠信，惜名節。是故同盟者得以守望相助，進退如一，莫逆於心。同盟黨派之間，彼此[MASK]。
保人者，受託于雙方顧主，牽繫買賣關係，以局外之身處理其間事務者也。顧主雙方多聘保人以求交易之公正，保人亦從中牟利。保人之制，早至魏晉隋唐已有所載，古商貿制度之完善據此可窺一二。保人與顧主之間，彼此[MASK]。
所謂忠臣，效忠于社稷者也，以鞠躬盡瘁、死而後已為己任，多為世之表率，歷代稱頌不絕。所謂逆賊，離君叛國者也，陷社稷於累卵之危、倒懸之急，世人為之不齒。忠臣與逆賊之間，彼此[MASK]。
主客之交，以互敬互重為本。為人主者，待人接物需完備周致，以求賓至如歸；為賓客者，亦需禮尚往來，以示謝忱。古之人倫禮制，亦見於主客相交之道也。主人與賓客之間，彼此[MASK]。
主奴者，主人為主而女婢為從也。主人有令，奴必應之，不可不從，以其必終於主人故也。奴之衣食起居，身家性命，全仰主意。主人和善體恤，則奴亦可豐足；主慳吝刻薄，則奴亦常縈困頓。然奴之私忠於其主也，不必係於主之慈暴，以世論其忠若犬故也。主奴之間，彼此[MASK]。
乳母之與嬰孩之家長，僱傭關係者也。家長或以嬌弱，或以矜貴，或以少乳，僱乳母以照料之。乳母當供乳於彼嬰幼，事其起居，乃或至教導培育。二者既系於僱傭，則信立為本而求職盡也。家長用人不疑，乳母方不墮其職也。乳母與家長之間，彼此[MASK]。
乳母之于嬰幼，保育者也。乳母非僅哺之以乳，且兼料其起居，助其成長，與其蒙學。長此以往，乳母或親愛嬰幼，嬰幼亦依戀乳母，此人情所致，本非乳母之分也。及長，嬰幼雖有生母，于禮當母事之，然愛信乳母，厚此而薄彼，亦時有之。雖然，若依通例，嬰幼年齒既長，無需哺乳，乳母之分即盡。更有甚者，就此別過，鮮復往來矣。乳母與嬰幼之間，彼此[MASK]。
凡職官者，常設正副。正者，統也。凡執掌決策